## Analyse results from the sea level projections NetCDF files

In [1]:
import xarray as xr
import numpy as np
import sys
sys.path.append('../code')
import func_misc as misc

/Users/dewilebars/anaconda3/envs/p37ds/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [5]:
AR5G85_ds = xr.open_dataset('../outputs/SeaLevelPDF_AR5_glo_rcp85.nc')
AR5G85_ds

<xarray.Dataset>
Dimensions:  (bin: 521, proc: 11, time: 95)
Coordinates:
  * time     (time) int64 2006 2007 2008 2009 2010 ... 2096 2097 2098 2099 2100
  * proc     (proc) object 'Global steric' 'Local ocean' ... 'sum anta.' 'Total'
  * bin      (bin) float64 -20.0 -19.0 -18.0 -17.0 ... 497.0 498.0 499.0 500.0
Data variables:
    MAT_RES  (time, proc, bin) float64 ...
Attributes:
    options:        Computations were done with the following options:: Local...
    source_file:    This NetCDF file was built from the Probabilistic Sea Lev...
    creation_date:  2020-02-10 11:46

In [3]:
Perc  = (1,5,10,17,20,50,80,83,90,95,99,99.5,99.9)
perc = misc.perc_df(AR5G85_ds.MAT_RES[-1,10,:], Perc, AR5G85_ds.bin)
perc

,values
percentiles,
1.0,46.0
5.0,53.0
10.0,57.0
17.0,61.0
20.0,63.0
50.0,74.0
80.0,85.0
83.0,87.0
90.0,92.0


In [4]:
AR5G85_ds = xr.open_dataset('../outputs/SeaLevelPDF_loc_rcp85.nc')
AR5G85_ds

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/dewilebars/Projects/Project_ProbSLR/SLProj/outputs/SeaLevelPDF_loc_rcp85.nc'